# 1. Define Functions

In [ ]:
from time import sleep
import csv
import requests

def dollar_string_to_int(string):
    try:
        return(float(string.replace(',','').replace('$','')))
    except:
        return string
    
def get_link_data(link):
    row = ['', '']
    if link and 'giveinkind' not in link:
        try:
            r = requests.get(link)
        except:
            return row
        if r.status_code == 200:
            text = r.text
            raised_start_index = text.index('m-progress-meter-heading') + 26
            raised_end_index = raised_start_index + text[raised_start_index:].find('<')
            raised = text[raised_start_index:raised_end_index]
            raised = dollar_string_to_int(raised)

            goal_start_index = raised_end_index + text[raised_end_index:].find('raised of ') + 10
            goal_end_index = goal_start_index + text[goal_start_index:].find(' ')
            goal = text[goal_start_index:goal_end_index]
            goal = dollar_string_to_int(goal)
            
            if isinstance(goal, float):
                pct_raised = raised / goal
                row = [f'${raised:,.0f}', f'{pct_raised:.0%}']
            else:
                row = [f'${raised:,.0f}', '']
    return row

# 2. Get Data From GoFundMe Links
Pulls Data from a CSV file named 'links.csv' and creates a csv file named 'results.csv'

In [ ]:
print('Running...')
data = []
with open('links.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    rows_done = 0
    for row in reader:
        family_name = row[0]
        link = row[1]
        row_data = get_link_data(link)
        data.append([family_name] + row_data)
        
        rows_done += 1
        # if rows_done == 5:
        #     break
        
        # sleeptime = random.uniform(1, 2.5)
        # sleep(sleeptime)
        print(rows_done, end=' ')
data = [['Name', 'Raised', 'Percent Raised']] + data[1:]

with open('result.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in data:
        writer.writerow(row)
        
print('All done!')